# Lab 05. Logistic Regression

In [2]:
# imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
# for reproducibility
torch.manual_seed(1)

In [4]:
# training data
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]] # (6, 2)
y_data = [[0], [0], [0], [1], [1], [1]] #(6,)

In [5]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [6]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


PyTorch has a `torch.exp()` function that resembles the exponential function.

In [8]:
print(f'e^1 ={torch.exp(torch.FloatTensor([1]))}')

e^1 =tensor([2.7183])


In [9]:
w = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [12]:
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(w) + b)))

In [13]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


In [14]:
print(f'1/(1+e^(-1))) = {torch.sigmoid(torch.FloatTensor([1]))}')

1/(1+e^(-1))) = tensor([0.7311])


In [16]:
hypothesis = torch.sigmoid(x_train.matmul(w) + b)

In [17]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
torch.Size([6, 1])


**Computing the Cost Function**

We want to measure the difference between `hypothesis` and `y_train`.

In [18]:
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)

**Full Training Procedure**

In [23]:
# init model
w = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# set optimizer
optimizer = optim.SGD([w, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(w)+b)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    # cost로 H(x)개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print(f'Epoch {epoch}/{nb_epochs} Cost: {cost.item()}')

Epoch 0/1000 Cost: 0.6931471824645996
Epoch 100/1000 Cost: 0.134722039103508
Epoch 200/1000 Cost: 0.08064313977956772
Epoch 300/1000 Cost: 0.05790001526474953
Epoch 400/1000 Cost: 0.0452997200191021
Epoch 500/1000 Cost: 0.037260960787534714
Epoch 600/1000 Cost: 0.03167249262332916
Epoch 700/1000 Cost: 0.027555925771594048
Epoch 800/1000 Cost: 0.024394338950514793
Epoch 900/1000 Cost: 0.021888310089707375
Epoch 1000/1000 Cost: 0.019852152094244957


**Evaluation**

In [24]:
hypothesis = torch.sigmoid(x_train.matmul(w) + b)
print(hypothesis[:5])

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01]], grad_fn=<SliceBackward0>)


We can change **hypothesis** (real number from 0 to 1) to **Binary predictions** (either 0 or 1) by comparing them to 0.5.

In [25]:
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])


In [28]:
correct_prediction = prediction.float() == y_train
print(correct_prediction[:5])

tensor([[True],
        [True],
        [True],
        [True],
        [True]])


**Higher Implementation with Class**

In [47]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [48]:
model = BinaryClassifier()

In [49]:
# set optimizer
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):
    
    # cost 계산
    hypothesis = model(x_train)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    # cost로 H(x)개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 10 == 0:
        
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        
        print(f'Epoch {epoch}/{nb_epochs} Accuracy: {accuracy*100}%')

Epoch 0/100 Accuracy: 50.0%
Epoch 10/100 Accuracy: 83.33333333333334%
Epoch 20/100 Accuracy: 83.33333333333334%
Epoch 30/100 Accuracy: 83.33333333333334%
Epoch 40/100 Accuracy: 83.33333333333334%
Epoch 50/100 Accuracy: 83.33333333333334%
Epoch 60/100 Accuracy: 83.33333333333334%
Epoch 70/100 Accuracy: 83.33333333333334%
Epoch 80/100 Accuracy: 100.0%
Epoch 90/100 Accuracy: 100.0%
Epoch 100/100 Accuracy: 100.0%
